# Multilabel Classification

### CS 640 Project

#### Ayush Shirsat, Sunitha Priyadarshini, Julie Park

In [ ]:
# Import all libraries
import tensorflow as tf
import keras
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, LeakyReLU, GlobalAveragePooling2D
from keras.models import Sequential, Input, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam
from keras.activations import relu, softmax, sigmoid
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras import regularizers
import matplotlib.pyplot as plt
import matplotlib
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Disable warning message of tensorflow
import numpy as np
import csv
import cv2

## Import Train data

In [ ]:
# path to train data
path1 = './train_re/'
X_data = []
Y_data1 = []
Y_data2 = []
Y_data3 = []

# load data
with open('annot_train.csv', 'r') as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
        X_data.append(row[0])
        Y_data1.append(row[1])
        Y_data2.append(row[2])
        Y_data3.append(row[3:13])        
csvFile.close()

X_data = X_data[1:len(X_data)]

X = np.array([np.array(cv2.imread(path1 + str(img),1)).flatten() for img in X_data],'f') 
X = X/255
X = X.reshape(X.shape[0], 128, 128, 3)
X = X.astype('float')

## Import Train Labels

In [ ]:
Y_data1 = np.array(Y_data1[1:len(Y_data1)])
Y_data2 = np.array(Y_data2[1:len(Y_data2)])
Y_data3 = np.array(Y_data3[1:len(Y_data3)])
Y_data1 = Y_data1.astype('float')
Y_data2 = Y_data2.astype('int')
Y_data3 = Y_data3.astype('int')

# Split data into validation and testing
X_val = X[0:6000]
X_train = X[6000:len(X)]

Y_data1_val = Y_data1[0:6000]
Y_data1 = Y_data1[6000:len(Y_data1)]

Y_data2_val = Y_data2[0:6000]
Y_data2 = Y_data2[6000:len(Y_data2)]

Y_data3_val = Y_data3[0:6000]
Y_data3 = Y_data3[6000:len(Y_data3)]

X_train /= 255
X_val /= 255

## Load ResNet-50 (add layers)

In [ ]:
# Run either Resnet or VGG

base_model=ResNet50(weights='imagenet',include_top=False, input_shape=(128, 128, 3)) # Uncomment For transfer learning
# base_model=ResNet50(weights = None, include_top=False, input_shape=(128, 128, 3)) # Uncomment For non-tranfer learning

# Added layers
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='sigmoid')(x) 
x=Dense(512,activation='sigmoid')(x) 
x=Dense(256,activation='sigmoid')(x) 

# output layers
x1 = Dense(units=1, kernel_regularizer=regularizers.l2(0.01))(x)
out1 = Activation(sigmoid)(x1)

x2 = Dense(units=1, kernel_regularizer=regularizers.l2(0.01))(x)
out2 = Activation(sigmoid)(x2)

x3 = Dense(units=10, kernel_regularizer=regularizers.l2(0.01))(x)
out3 = Activation(sigmoid)(x3)

for layer in base_model.layers:
    layer.trainable=False

## Load VGG16 (add layers)

In [ ]:
# Run either Resnet or VGG

base_model=VGG16(weights='imagenet',include_top=False, input_shape=(128, 128, 3)) # Uncomment For transfer learning
# base_model=VGG16(weights = None, include_top=False, input_shape=(128, 128, 3)) # Uncomment for non-tranfer learning

# Added layers
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='sigmoid')(x) 
x=Dense(512,activation='sigmoid')(x) 
x=Dense(256,activation='sigmoid')(x) 

# output layers
x1 = Dense(units=1, kernel_regularizer=regularizers.l2(0.01))(x)
out1 = Activation(sigmoid)(x1)

x2 = Dense(units=1, kernel_regularizer=regularizers.l2(0.01))(x)
out2 = Activation(sigmoid)(x2)

x3 = Dense(units=10, kernel_regularizer=regularizers.l2(0.01))(x)
out3 = Activation(sigmoid)(x3)

for layer in base_model.layers:
    layer.trainable=False

## Start Training

In [ ]:
# Initialize parameters
model = Model(inputs=base_model.input, outputs=[out1,out2,out3])
adam = Adam(lr = 0.001)
model.compile(loss=['mean_squared_error','binary_crossentropy','binary_crossentropy'], optimizer=adam, metrics=['accuracy'])

# Initialize callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.00001)
es = EarlyStopping(monitor='val_loss', patience=3)

# Start training the model
hist = model.fit(X_train, [Y_data1,Y_data2,Y_data3] , epochs=15, validation_data =(X_val, [Y_data1_val,Y_data2_val,Y_data3_val]) , batch_size=256, callbacks=[reduce_lr, es])

## Plot Train & Validation loss

In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss'] 

plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(hist.history["loss"], label="loss")
plt.plot(hist.history["val_loss"], label="val_loss")
plt.plot( np.argmin(hist.history["val_loss"]), np.min(hist.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.legend();

## Load Test data

In [ ]:
X_test = []
Y_test1 = []
Y_test2 = []
Y_test3 = []

# Load test data
with open('annot_test.csv', 'r') as csvFile:
    reader2 = csv.reader(csvFile)
    for row2 in reader2:
        X_test.append(row2[0])
        Y_test1.append(row2[1])
        Y_test2.append(row2[2])
        Y_test3.append(row2[3:13])
csvFile.close()

# Path for test data
path2 = './test_re/'
X_test = X_test[1:len(X_test)]

Xte = np.array([np.array(cv2.imread(path2 + str(img2),1)).flatten() for img2 in X_test],'f') 
Xte = Xte/255
Xte = Xte.reshape(Xte.shape[0], 128, 128, 3)
Xte = Xte.astype('float')
X_test = Xte

Y_test1 = np.array(Y_test1[1:len(Y_test1)])
Y_test2 = np.array(Y_test2[1:len(Y_test2)])
Y_test3 = np.array(Y_test3[1:len(Y_test3)])
Y_test1 = Y_test1.astype('float')
Y_test2 = Y_test2.astype('int')
Y_test3 = Y_test3.astype('int')

## Prediction

In [ ]:
score = model.evaluate(X_test,[Y_test1,Y_test2,Y_test3])
print(score[4:7])

# i is what test sample to predict
i=0
temp = X_test[i].reshape(1,128,128,3)
pred = model.predict(temp)
print(Y_test1[i],Y_test2[i],Y_test3[i],'\n', pred)